# Waterprofiel & Residual Alkalinity (RA) – basisrekenmodel

**Doel.** Een didactisch notebook om (i) een **waterprofiel** te berekenen na zouttoevoegingen en (ii) de **Residual Alkalinity (RA)** te schatten.

> Vereenvoudigd model: geen activiteit/ionsterkte, geen CO₂-evenwicht, geen neerslagkinetiek (CaCO₃), geen mout-specifieke titratiecurves.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

V_L = 20.0  # [L]

start = {
    "Ca": 35.0,
    "Mg": 8.0,
    "Na": 15.0,
    "Cl": 30.0,
    "SO4": 40.0,
    "HCO3": 120.0,
    "Alk_as_CaCO3": 100.0
}

add_g = {
    "CaCl2_2H2O": 0.0,
    "CaSO4_2H2O": 0.0,
    "MgSO4_7H2O": 0.0,
    "NaCl": 0.0,
    "NaHCO3": 0.0
}


In [ ]:

MW = {
    "Ca": 40.078, "Mg": 24.305, "Na": 22.990, "Cl": 35.45,
    "S": 32.065, "O": 15.999, "H": 1.008, "C": 12.011
}

def mw_SO4():
    return MW["S"] + 4*MW["O"]

def mw_HCO3():
    return MW["H"] + MW["C"] + 3*MW["O"]

def mw_H2O():
    return 2*MW["H"] + MW["O"]

salts = {}

salts["CaCl2_2H2O"] = {
    "MW": MW["Ca"] + 2*MW["Cl"] + 2*mw_H2O(),
    "Ca": MW["Ca"],
    "Cl": 2*MW["Cl"]
}

salts["CaSO4_2H2O"] = {
    "MW": MW["Ca"] + mw_SO4() + 2*mw_H2O(),
    "Ca": MW["Ca"],
    "SO4": mw_SO4()
}

salts["MgSO4_7H2O"] = {
    "MW": MW["Mg"] + mw_SO4() + 7*mw_H2O(),
    "Mg": MW["Mg"],
    "SO4": mw_SO4()
}

salts["NaCl"] = {
    "MW": MW["Na"] + MW["Cl"],
    "Na": MW["Na"],
    "Cl": MW["Cl"]
}

salts["NaHCO3"] = {
    "MW": MW["Na"] + mw_HCO3(),
    "Na": MW["Na"],
    "HCO3": mw_HCO3()
}

def delta_ppm_from_salt(salt_key, grams, V_L):
    if grams == 0:
        return {}
    info = salts[salt_key]
    MW_salt = info["MW"]
    out = {}
    for ion, ion_mass in info.items():
        if ion == "MW":
            continue
        out[ion] = (grams * (ion_mass / MW_salt) * 1000.0) / V_L
    return out

delta = {k: 0.0 for k in ["Ca","Mg","Na","Cl","SO4","HCO3"]}

for salt_key, grams in add_g.items():
    d = delta_ppm_from_salt(salt_key, grams, V_L)
    for ion, val in d.items():
        delta[ion] += val

profile = {ion: start.get(ion,0.0) + delta.get(ion,0.0)
           for ion in ["Ca","Mg","Na","Cl","SO4","HCO3"]}

profile["Alk_as_CaCO3"] = start["Alk_as_CaCO3"]

profile


In [ ]:

def residual_alkalinity(alk_as_caco3, Ca_mgL, Mg_mgL):
    return alk_as_caco3 - (Ca_mgL/3.5 + Mg_mgL/7.0)

RA = residual_alkalinity(profile["Alk_as_CaCO3"], profile["Ca"], profile["Mg"])

print("Residual Alkalinity (RA):", round(RA,1), "mg/L as CaCO3")


In [ ]:

ions = ["Ca","Mg","Na","Cl","SO4","HCO3"]
start_vals = [start.get(i,0.0) for i in ions]
new_vals = [profile.get(i,0.0) for i in ions]

x = np.arange(len(ions))
w = 0.38

plt.figure(figsize=(9,4.8))
plt.bar(x - w/2, start_vals, width=w, label="Start")
plt.bar(x + w/2, new_vals, width=w, label="Na toevoeging")
plt.xticks(x, ions)
plt.ylabel("mg/L (ppm)")
plt.title("Waterprofiel – start vs. na zouttoevoeging")
plt.grid(True, axis="y", alpha=0.25)
plt.legend()
plt.tight_layout()
plt.show()
